# Area selection based on Wind Spead

All automatic stations in Northeast region of Brazil will be analysed

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import distance

import os

%load_ext autoreload
%autoreload 2
from data_preprocessing import initial_data_preprocessing

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)


In [25]:
df_catalog_automatic_stations = pd.read_excel('./../../data/automatic_stations_catalog.xlsx')[[
                                                        'DC_NOME','SG_ESTADO','CD_SITUACAO',
                                                        'VL_LATITUDE','VL_LONGITUDE','VL_ALTITUDE',
                                                        'DT_INICIO_OPERACAO','CD_ESTACAO']]

df_catalog_automatic_stations = df_catalog_automatic_stations.query("CD_SITUACAO != 'Pane'")

northeast_stations = os.listdir('./../../data/raw_data')
# northeast_stations = [x.split('.')[0].replace('dados_','') for x in northeast_stations]
df_northeast_stations = pd.DataFrame(data=northeast_stations,columns=['STATION_FILE_NAME'])
df_northeast_stations['CD_ESTACAO'] = [x.split('.')[0].replace('dados_','').split('_H_')[0] for x in northeast_stations]
df_northeast_stations = df_northeast_stations.merge(df_catalog_automatic_stations,on='CD_ESTACAO')

In [24]:
for ii,station in df_northeast_stations.iterrows():
    df = pd.read_csv("./../../data/raw_data/" + station['STATION_FILE_NAME'],header=9,sep=';')
    print(f"Analysing {station['CD_ESTACAO']} - {station['DC_NOME']} station")
    df = initial_data_preprocessing(df)
    break

Analysing A424 - IRECE station
Dataset initial size:  (90072, 23)
Dataset final size:  (39948, 23)


In [46]:
df.isna().sum().sum() #/ (df.shape[0] * (df.shape[1]- 4)) * 100

16275

In [49]:
df.dropna().shape

(36335, 23)

In [51]:
df_s = df[['WIND_SPEED_120m_ms']].agg([np.mean,np.median,np.std]) 

wind_statistics = {
    'WIND_SPEED_mean': df['WIND_SPEED_ms'].mean(),
    'WIND_SPEED_120m_mean': df['WIND_SPEED_120m_ms'].mean(),
    'WIND_SPEED_median': df['WIND_SPEED_ms'].median(),
    'WIND_SPEED_120m_median': df['WIND_SPEED_120m_ms'].median(),
    'WIND_SPEED_std': df['WIND_SPEED_ms'].std(),
    'WIND_SPEED_120m_std': df['WIND_SPEED_120m_ms'].std(),
    'MINIMUM_DATE': df['DATE_MEASUREMENT'].min(),
    'DATASET_SIZE': df['WIND_SPEED_ms'].shape[0],
    'NON_NULL_DATASET_SIZE': df.dropna().shape[0],
    'NULL_CELLS_QUANTITY': df.isna().sum().sum(),

}
wind_statistics

{'WIND_SPEED_mean': 2.6444552918794435,
 'WIND_SPEED_120m_mean': 4.1263475348402165,
 'WIND_SPEED_median': 2.6,
 'WIND_SPEED_120m_median': 4.056980514486104,
 'WIND_SPEED_std': 1.540014439902529,
 'WIND_SPEED_120m_std': 2.403003297966074,
 'MINIMUM_DATE': '2017-09-19',
 'DATASET_SIZE': 39948,
 'NULLS_QUANTITY': 16275,
 'NON_NULL_DATASET_SIZE': 36335}

In [33]:
df_s

,WIND_SPEED_ms,WIND_SPEED_120m_ms
mean,2.644455,4.126348
median,2.600000,4.056981
std,1.540014,2.403003


In [ ]:
def extract_wind_speed_statistics(df: pd.DataFrame) -> dict:
    '''
    Extracts relevant information about the wind speed from the automatic stations.
    
    Parameters
    ----------
    df: Preprocessed dataframe containing the historic information about the wind speed and direction.
    
    Returns 
    ----------
    statistics: dict containing the mean, median, std and minimum date available for the wind speed.

    '''

    df = df[['WIND_SPEED_ms','WIND_SPEED_120m_ms']].agg([np.mean,np.median,np.std]) #pivotar essa tabela para achatar esse dataframe, transformar tudo em uma linha so com o nome da estacao.


In [4]:
df_automatic_stations['SG_ESTADO'].unique()

states = ['BA','']

array(['MT', 'MS', 'DF', 'MG', 'ES', 'RS', 'TO', 'GO', 'BA', 'RJ', 'SC',
       'PA', 'SP', 'MA', 'SE', 'RR', 'CE', 'PE', 'AL', 'PB', 'PI', 'RN',
       'PR', 'AM', 'AP', 'AC', 'RO'], dtype=object)